# import library

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from underthesea import sent_tokenize, word_tokenize, pos_tag
from selenium.webdriver.support.ui import WebDriverWait
import re
from keras.utils import pad_sequences
from keras.models import load_model
import yaml
import numpy as np
import pandas as pd
from selenium.common.exceptions import TimeoutException

# load_model

In [23]:
model = load_model("model_comment.tf")

# read_file_dictionary_for_model

In [24]:
def doc_tep_yaml(duong_dan):
    try:
        with open(duong_dan, 'r', encoding='utf-8') as file:
            du_lieu = yaml.safe_load(file)
        return du_lieu
    except FileNotFoundError:
        print(f"Tệp {duong_dan} không tồn tại.")
        return None
    except Exception as e:
        print(f"Có lỗi xảy ra khi đọc tệp YAML: {str(e)}")
        return None



loaded_data = doc_tep_yaml('dictionary_yaml.yaml')

# get_name_phone

In [25]:
def get_name_phone(driver):
    element = driver.find_element(By.XPATH, "//div[@class='cate-main-container']/section[@id='categoryPage']/div[3]")
    element1 = element.find_elements(By.XPATH, "//ul[@class='listproduct']/li/a[1]")
    return element1

In [26]:
url = "https://www.dienmayxanh.com/dien-thoai"

In [27]:

driver = webdriver.Chrome()
driver.get(url)

for _ in range(5):

    element_click = driver.find_element(By.XPATH, "//div[@class='cate-main-container']/section[@id='categoryPage']/div[3]/div[2]")
    element_click = element_click.find_element(By.TAG_NAME, 'a')
    element_click.click()
    time.sleep(5)

name_phone = get_name_phone(driver)
list_smartphone = []
for i in name_phone:
    list_smartphone.append(i.get_attribute('href'))

driver.quit()

In [28]:
def remove_utm(url):
    return url.split('?')[0]

list_smartphone = [remove_utm(url) for url in list_smartphone]
print(list_smartphone)

['https://www.dienmayxanh.com/dien-thoai/iphone-15-pro-max', 'https://www.dienmayxanh.com/dien-thoai/samsung-galaxy-s23-ultra', 'https://www.dienmayxanh.com/dien-thoai/oppo-reno11-f-5g-phien-ban-chuan', 'https://www.dienmayxanh.com/dien-thoai/xiaomi-redmi-note-13', 'https://www.dienmayxanh.com/dien-thoai/vivo-y100-8-128', 'https://www.dienmayxanh.com/dien-thoai/realme-c65-8-128', 'https://www.dienmayxanh.com/dien-thoai/samsung-galaxy-a55-5g-128gb', 'https://www.dienmayxanh.com/dien-thoai/iphone-13', 'https://www.dienmayxanh.com/dien-thoai/samsung-galaxy-m54', 'https://www.dienmayxanh.com/dien-thoai/xiaomi-redmi-note-13-pro', 'https://www.dienmayxanh.com/dien-thoai/oppo-a58', 'https://www.dienmayxanh.com/dien-thoai/realme-note-50-64gb', 'https://www.dienmayxanh.com/dien-thoai/samsung-galaxy-s23-fe', 'https://www.dienmayxanh.com/dien-thoai/samsung-galaxy-a35-5g-128gb', 'https://www.dienmayxanh.com/dien-thoai/vivo-y36-128gb', 'https://www.dienmayxanh.com/dien-thoai/oppo-a18', 'https://www

# get_comment

In [2]:
def classify_topic(sentence):
    keywords_phone = {"điện thoại", "smartphone", "máy", 'may', 'dien thoai', 'man', 'màn'}
    keywords_battery = {"pin", "thời lượng pin", "sạc", 'dung lượng', 'dung luong'}
    keywords_service = {"dịch vụ", "bảo hành", "khách hàng", 'dv', 'kh', 'nhân viên', 'nhan vien', 'Mv'}

    words = set(sentence.lower().split())
    if words & keywords_phone:
        return "Điện thoại"
    elif words & keywords_battery:
        return "Pin"
    elif words & keywords_service:
        return "Dịch vụ"
    
    # Sử dụng POS tagging để tìm danh từ

    tagged_words = pos_tag(sentence)
    for i in range(len(tagged_words) - 1):
        word, tag = tagged_words[i]
        next_word, next_tag = tagged_words[i + 1]
        if tag == 'N' and next_tag == 'A':  # Tìm cặp danh từ và tính từ liền nhau
            return word, next_word

    return "Không xác định"

In [3]:
import matplotlib.pyplot as plt

def plot_pie_chart(data, name_file):
    # Tính toán tổng số lượng của tất cả các nhãn trong danh sách
    total_count = len(data)
    
    # Tạo một từ điển để lưu trữ số lượng của mỗi nhãn
    label_counts = {}
    for label in data:
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1
    
    # Tính phần trăm của mỗi nhãn
    labels = []
    sizes = []
    for label, count in label_counts.items():
        labels.append(f'{label} ({count})')  # Thêm số lượng vào nhãn
        sizes.append(count / total_count * 100)
    
    # Vẽ biểu đồ tròn
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)  # Sử dụng autopct để hiển thị phần trăm và số lượng
    plt.axis('equal')  # Đảm bảo biểu đồ tròn
    plt.title(f'{name_file.split(".")[0]} and total {total_count}')
    plt.show()


In [4]:
def proces_input(data, word_to_id, maxlen):
  data = word_tokenize(re.sub(r'[^\w\s]', '', data).lower())
  inputs = [word_to_id[word] if word in word_to_id.keys() else 1 for word in data]
  inputs = pad_sequences([inputs], maxlen=maxlen,padding='post',truncating='post')
  return inputs

dic1 = {0: 'positive', 1: 'negative'}
def pre(model, text, d, loaded_data):

  inputs = proces_input(text, loaded_data, 307)
  predic = model(inputs)
  predic = np.argmax(predic, axis=-1)
  return d[predic[0]]


In [ ]:
def get_text(driver, url):
    data_all = []
    elem1 = driver.find_elements(By.XPATH, "/html/body/div[6]/div/div[2]/div/div/div[7]/div[2]/div/a")
    
    if len(elem1) > 1:
        number_click = int(elem1[-2].text)
        
        for number in range(number_click):
            driver.get(f"{url}/danh-gia?page={number + 1}")

            try:
                elem3 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[6]/div/div[2]/div/div/div[7]")))
                comments = elem3.find_elements(By.CLASS_NAME, 'cmt-txt')
                for j in comments:
                    if j.text:
                        text1 = sent_tokenize(j.text)
                        if len(text1) > 1:
                            for s1 in text1:
                                if len(word_tokenize(s1)) > 3:
                                    a1 = pre(model, s1, dic1, loaded_data)
                                    dt1 = [s1, classify_topic(s1), a1, f"{url.split('/')[-1]}"]            
                                    data_all.append(dt1)
                        else:
                            a1 = pre(model, j.text, dic1, loaded_data)
                            dt1 = [j.text, classify_topic(j.text), a1, f"{url.split('/')[-1]}"]            
                            data_all.append(dt1)
                    else:
                        break
                        
            except NoSuchElementException:
                print(f"{url.split('/')[-1]} không tìm thấy bình luận nào cả")
    else:
        try:
            elem4 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[6]/div/div[2]/div/div/div[7]")))
            comments1 = elem4.find_elements(By.CLASS_NAME, 'cmt-txt')
            for i in comments1:
                if i.text:
                    text1 = sent_tokenize(i.text)
                    if len(text1) > 1:
                        for s in text1:
                            if len(word_tokenize(s)) > 3:
                                a1 = pre(model, s, dic1, loaded_data)
                                dt1 = [s, classify_topic(s), a1, f"{url.split('/')[-1]}"]            
                                data_all.append(dt1)
               
                    else:
                        a1 = pre(model, i.text, dic1, loaded_data)
                        dt1 = [i.text, classify_topic(i.text), a1, f"{url.split('/')[-1]}"]            
                        data_all.append(dt1)
                            
                else:
                    break
        
        except NoSuchElementException:
                print(f"{url.split('/')[-1]} không tìm thấy bình luận nào cả")

    return data_all


In [5]:
# for link in list_smartphone:

def check_review(driver, url):
    try:
        driver.set_page_load_timeout(10)
        driver.get(url)
        try:
            
            el = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/section[1]/div[3]/div[1]")))
            el = el.find_element(By.CLASS_NAME, "box-flex")
            el = el.find_element(By.TAG_NAME, 'a')
            el.click()
                
            dt = get_text(driver, url)
            return dt
        except NoSuchElementException:
            print(f"{url.split('/')[-1]} không có bình luận")
    except TimeoutException:
        pass


In [ ]:
driver = webdriver.Chrome()
data_all1 = []
for urls in list_smartphone:
    dtt = check_review(driver, urls)
    if dtt is not None:
        data_all1.extend(dtt)
df = pd.DataFrame(data_all1, columns=['sentence', 'title', 'prediction', 'name phone'])
df.to_excel('all_phone.xlsx', index=False)
driver.quit()

# read_file_name_phone

In [6]:
# def write_links_to_file(links, filename):
#     with open(filename, 'w') as file:
#         for link in links:
#             file.write(link + '\n')

# Đọc từng dòng trong file
# def read_links_from_file(filename):
#     with open(filename, 'r') as file:
#         links = file.readlines()
#         links = [link.strip() for link in links]  # Xóa ký tự xuống dòng
#     return links

# filename = "links.txt"

# Viết danh sách các đường link vào file
# write_links_to_file(list_smartphone, filename)

# Đọc từng dòng trong file và in ra màn hình


In [7]:
# list_smartphone = []
# read_links = read_links_from_file(filename)
# for link in read_links:
#     list_smartphone.append(link)
# print(list_smartphone)

# draw a chart

In [ ]:
# import pandas as pd
# file_path = "all_phone.xlsx"
# df = pd.read_excel(file_path)

# df = df['prediction']
# plot_pie_chart(df, file_path)

In [ ]:
# import pandas as pd
# file_path = "samsung.xlsx"
# df = pd.read_excel(file_path)

# df = df['prediction']
# plot_pie_chart(df, file_path)